# Off Policy MC 
Does not work as Q is not changing.

In [ ]:
import numpy as np
from gridworld import GridworldEnv

env = GridworldEnv()

In [ ]:
def mc_control_off_policy(env, num_episodes, discount=1.0, debug=False):
    def argmax_a(arr):
        """
        Return idx of max element in an array.
        Break ties uniformly.
        """
        max_idx = []
        max_val = float("-inf")
        for idx, elem in enumerate(arr):
            if elem == max_val:
                max_idx.append(idx)
            elif elem > max_val:
                max_idx = [idx]
                max_val = elem
        return np.random.choice(max_idx)

    def get_b(state, Q, eps=0.2):
        if np.random.random() < eps:
            A = np.ones(env.nA, dtype=float) / env.nA
        else:
            A = np.zeros(env.nA)
            A[argmax_a(Q[state])] = 1
        return A

    Q = np.zeros((env.nS, env.nA)) + 0
    C = np.zeros((env.nS, env.nA))

    for i_episode in range(1, num_episodes + 1):
        if debug:
            if i_episode % 100 == 0:
                print("\rEpisode {}/{}.".format(i_episode, num_episodes))

        state, _ = env.reset()
        episode = []
        while True:
            probs = get_b(state, Q)
            action = np.random.choice(len(probs), p=probs)
            next_state, reward, done, _, _ = env.step(action)
            episode.append((state, action, reward))
            if done:
                break
            state = next_state

        G = 0.0
        W = 1.0

        for t in range(len(episode))[::-1]:
            state, action, reward = episode[t]
            G = discount * G + reward
            C[state, action] += W
            Q[state, action] += (W / C[state, action]) * (G - Q[state, action])
            if action != argmax_a(Q[state]):
                break
            prob = get_b(state, Q)[action]
            if prob == 0:
                break
            W = W * (1 / prob)

    V = np.zeros(env.nS)
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        best_action = argmax_a(Q[s])
        policy[s] = best_action
        V[s] = Q[s][best_action]

    return Q, policy, V

In [ ]:
Q_pi, policy, V_pi = mc_control_off_policy(env, num_episodes=1000, debug=True)

In [ ]:
Q_pi

In [ ]:
policy

In [ ]:
V_pi